Cyclistic Bike Share Data Analysis

Cyclistic is a bike share company in chicago which has asked me to perform data analysis to advise their marketing efforts.  The director of marketing believes they must maximize annual memberships in order to generate greater profits from the bike-share system.  My audience is the internal stakeholders, including the marketing director, other marketing employees, as well as other analysts in my department.  My analysis will help our marketing team identify who to advertise to for the maximization of annual memberships. 

The data provided by Cyclistic is organized byn month in indivivual .csv files. These files were likely pulled from a table in the cyclistic database.  It is organized by ride, with each ride having a unique ride ID.  I believe this data is reliable considering it came directly from the stakeholder's database.  This information would have been collected automatically by the bike stations so it is unlikely there are errors in the data.

In [ ]:
import pandas as pd
import os
import numpy as np
import datetime as dt
import math
import seaborn as sns
import matplotlib.pyplot as plt


for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

I want to set my default path to the location of my dataset so that I can acces their names.  From here I will make a list of the .csv file names:

In [ ]:
os.chdir('/kaggle/input/cyclistic-trips-202108-to-202207')
filelist = os.listdir()

By creating a dataframe and appending all 12 months of data to the same frame, I can run operations on the whole dataset rather than just one month at a time.  A preview of this new dataframe is shown below:

In [ ]:
tempFrame = pd.DataFrame()
frame = pd.DataFrame()
for file in filelist: 
    tempFrame = pd.read_csv(file)
    frame = frame.append(tempFrame)

frame.head()

To start that data cleaning process, I will need some more information about the dataset. Let's begin by finding out how many rows and collumns there are:

In [ ]:
frame.shape

There are 12 collumns and 5,901,463 rows.  That's a lot of trips, each one with their own unique ride_id. This data was found already in .csv format and I did no have to pull it from a database. I would assume that ride_id is the primary key and all this information is stored in a table of ride data.

We must ensure our data is in the correct operable data type.

Here we will be able to see the datatypes of attributes:

In [ ]:
frame.info()

The attributes 'started_at' and 'ended_at' use the data type object, but should be cast to a date format:

In [ ]:
frame['started_at'] = pd.to_datetime(frame['started_at'])
frame['ended_at'] = pd.to_datetime(frame['ended_at'])
frame.info()

Now that our ride start and end data is formatted we can create new collumns of information from these attributes.

I will the weekday of a ride can be found by matching the dayofweek index to the associated list of weekdays:

In [ ]:
day = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
frame['weekday'] = [day[i] for i in frame['started_at'].dt.dayofweek]

And I will find the difference between the start and end times to calculate a ride duration.  The resulting timedelta64 value is converted into hours and cast to float:

In [ ]:
td = frame ['ended_at'] - frame['started_at']
frame['duration_minutes'] = td / np.timedelta64(1, 'm')

The ride distance can be found using the distance formula and lattutude & longitude location data:
d=√((x_2-x_1)²+(y_2-y_1)²)
This distance is only an approximation due to the nature of lattitude and longitude conversions

In [ ]:
latdif = frame['start_lat']*69-frame['end_lat']*69
lngdif = frame['start_lng']*54.6-frame['end_lng']*54.6 
distbase = latdif**2 + lngdif**2
frame['distance'] = distbase**.5

Now that both time in hours and approximate distance in hours are both recorded as float, we can calculate the average speed of the ride:

In [ ]:
frame['avg_speed_mph'] = 60*frame['distance']/frame['duration_minutes']

Finally we will add a month collumn:

In [ ]:
frame['month'] = pd.DatetimeIndex(frame['started_at']).month

Now that all the necessary collumns have been added, it is time to clean the data.

Here we can see all the entries with a null value:

In [ ]:
frame.isnull().sum()

After taking a closer look, there are many rides with either a start location and no end location, and end location and no start location, or no start or end location.  This probalby happens when: a bike is taken out for maintainence, a bike is put back in after maintainance, a bike is lost or damaged, a bike is not returned properly, or there is an error in recording the ride information.

In any of these cases where location data is missing, we want to exclude the ride from our analysis:

In [ ]:
cleanframe = frame.dropna()
cleanframe.isna().sum()

Before our data cleaning is finished we shoud see if there are any outliers that need to be left out:

In [ ]:
cleanframe.describe()

It looks like there are some negative duration values, rides multiple days long, 0 mile rides, and some over 800 miles.  In this analysis I will only include rides between 30 seconds and 8 hours and between .1 and 100 miles.

In [ ]:
conditions = (cleanframe['duration_minutes'] < .5)|(cleanframe['duration_minutes'] > 480)|(cleanframe['distance'] < .1)|(cleanframe['distance'] > 100)
finframe = cleanframe[~conditions]
finframe.describe()

In [ ]:
finframe.shape

Now we have a dataset that only includes the useable ride entries.  There are now 4,312,244 columns and 17 rows.

The data has been properly formatted and cleaned so it is time to move onto the analysis on the remaining 4.3 million rides.
Here are some of the things I want to look at in my analysis: Member vs Casual total distance, average distance, speed, total use, bike type, monthly use.

In [ ]:
member_type =finframe["member_casual"].value_counts()
member_type

In [ ]:
plt.pie(member_type.values, labels =member_type.index,autopct='%1.1f%%')
plt.title("Share of rides by user type")
plt.legend(member_type.index)
plt.show()

More member rides than casual

In [ ]:
casual_type = finframe.pivot_table(index =['member_casual','rideable_type'],
                       values = ['duration_minutes'],
                       aggfunc ={'mean'})
print(casual_type)
df_length_day = pd.DataFrame({'Bike Type':casual_type.reset_index()["rideable_type"],
                                     'Ride Duration' : casual_type.reset_index()['duration_minutes']['mean']})

In [ ]:
speed = finframe.pivot_table(index =['member_casual'],
                       values = ['avg_speed_mph'],
                       aggfunc ={'mean'})
print(speed)

In [ ]:
plot = speed.T.plot(kind='bar', ylabel='Avg Speed (mph)')
plot.axes.get_xaxis().set_visible(False)

In [ ]:
speed_bike = finframe.pivot_table(index =['member_casual','rideable_type'],
                       values = ['avg_speed_mph'],
                       aggfunc ={'mean'})
df_speed_bike = pd.DataFrame({'Bike Type':speed_bike.reset_index()["rideable_type"],
                                     'Avg Speed' : speed_bike.reset_index()['avg_speed_mph']['mean']})
df_speed_bike

In [ ]:
sns.barplot(x='Bike Type', y= 'Avg Speed',data = df_speed_bike, hue = speed_bike.reset_index()['member_casual'])

Across both types member travel faster

In [ ]:
count_bike = finframe.pivot_table(index =['member_casual','rideable_type'],
                       values = ['ride_id'],
                       aggfunc ={'count'})
df_count_bike = pd.DataFrame({'Bike Type':count_bike.reset_index()["rideable_type"],
                                     'Number of Rides' : count_bike.reset_index()['ride_id']['count']})
df_count_bike

In [ ]:
sns.barplot(x='Bike Type', y= 'Number of Rides',data = df_count_bike, hue = count_bike.reset_index()['member_casual'])

Members prefer classic bike

In [ ]:
plot = count_bike.plot.pie(y='ride_id', figsize=(5, 5), title = 'Number of Rides')

In [ ]:
ride_length = finframe.pivot_table(index =['member_casual'],
                       values = ['duration_minutes'],
                       aggfunc ={'mean'})
print(ride_length)

In [ ]:
plot = ride_length.T.plot(kind='bar', ylabel='Avg Ride Duration (minutes)')
plot.axes.get_xaxis().set_visible(False)

Members take shorter rides

In [ ]:
length_day = finframe.pivot_table(index =['member_casual','weekday'],
                       values = ['duration_minutes'],
                       aggfunc ={'mean'})
print(length_day)
df_length_day = pd.DataFrame({'weekday':length_day.reset_index()["weekday"],
                                     'Ride Duration' : length_day.reset_index()['duration_minutes']['mean']})

In [ ]:
sns.barplot(x='weekday', y= 'Ride Duration',data = df_length_day, hue = length_day.reset_index()['member_casual'])

Members use the bike for the same duration regardless of day

In [ ]:
total_length_day = finframe.pivot_table(index =['member_casual','weekday'],
                       values = ['duration_minutes'],
                       aggfunc ={'count'})
print(total_length_day)
df_total_length_day = pd.DataFrame({'Weekday':total_length_day.reset_index()["weekday"],
                                     'Total Ride Minutes' : total_length_day.reset_index()['duration_minutes']['count']})

In [ ]:
sns.barplot(x='Weekday', y= 'Total Ride Minutes',data = df_total_length_day, hue = total_length_day.reset_index()['member_casual'])

Memeber ride more during weekdays than weekends

In [ ]:
month_use = finframe.pivot_table(index =['member_casual','month'],
                       values = ['ride_id'],
                       aggfunc ={'count'})
print(month_use)

In [ ]:
sns.barplot(x =month_use.reset_index()['month'], y= month_use.reset_index()['ride_id']['count'],
            hue =  month_use.reset_index()['member_casual']);

In [ ]:
distance = finframe.pivot_table(index =['member_casual'],
                       values = ['distance'],
                       aggfunc ={'mean'})
print(distance)
df_distance = pd.DataFrame({'Member Type':distance.reset_index()["member_casual"],
                                     'Avg Distance Traveled' : distance.reset_index()['distance']['mean']})

In [ ]:
sns.barplot(x='Member Type', y= 'Avg Distance Traveled',data = df_distance, hue = distance.reset_index()['member_casual'])

In [ ]:
distance_bike = finframe.pivot_table(index =['member_casual','rideable_type'],
                       values = ['distance'],
                       aggfunc ={'mean'})
print(distance_bike)
df_distance_bike = pd.DataFrame({'Bike Type':distance_bike.reset_index()["rideable_type"],
                                     'Avg Distance Traveled' : distance_bike.reset_index()['distance']['mean']})

In [ ]:
sns.barplot(x='Bike Type', y= 'Avg Distance Traveled',data = df_distance_bike, hue = distance_bike.reset_index()['member_casual'])

Members are more sensetive to seasonal changes and ride most in warmer months

Alltogether, members behave much different that casual riders.  Now I have to relate this back to the originial question:
Which riders should be marketed to to maximise anual memberships.

The data shows that members ride faster, ride shorter distances, ride more during weekends rather than weekdays, ride more during summer months, ride significantly more on the classic bike rather than electric, and take up most of the total rides.


Analyst Reccomendation:

Focus marketing efforts on commuting riders who bike quickly and go shorter distances.  These promotions should take place before and during June-Aug.
More demographic information would improve analysis efforts.

Reflection:

When looking at other similar projects the most overlooked sections is data cleaning.  Most people completing the Google Data Analytics Certificate forgot to remove recording errors or large outlies like negative or great distances and ride durations.  Aditionally most other analysts did not include distance and speed data and only relied on duration information. Although it was not necessary here is is still an important factor to consider.

Some things that I wanted to inclucde but was incapable: geographic analysis based on station and latitude & longitude data.  Map of member vs casual rides displayed over chicago city map.

Overall I enjoyed this project and look foreward to my work ahead